<a href="https://colab.research.google.com/github/haligene109/Eugen/blob/main/tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we are going to cover some of the most fundamental concepts of tensor using tensorflow

More specifically, we're  going to cover:
* Introduction to tensors
*Getting information from tensorstor
*Manipulating tensors
*Tensors & Numpy
* Using @tf.function (a way to speed up your regular python functions)
* Using GPUs with TensorFlow(orTPUs)
* Exercise  to try myself

# Introduction to Tensors

In [1]:
# Import Tensorflow
import tensorflow as tf
print(tf.__version__)

2.14.0


tf.constant Variable

In [2]:
# Create tensors with ft.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Check the number of dimensions  of a tensor (ndim stands for number of dimensions)
scalar.ndim


0

In [4]:
# create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
vector.ndim

1

In [6]:
# create a matrix (a matrix has more than 1 dimension)
matrix = tf.constant([[10, 7], [7, 10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [7]:
matrix.ndim

2

In [8]:
# create another matrix
another_matrix = tf.constant([[10., 7.],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16)# specify the data type parameter
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
# what is the number of dimensions of another_matrix
another_matrix.ndim


2

In [10]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                       [[7, 8, 9],
                       [10, 11, 12]],
                       [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [11]:
tensor.ndim

3

**tf.Variable**. Variables are created and tracked via the tf.Variable class. A tf.Variable represents a tensor whose value can be changed by running ops on it. Specific ops allow you to read and modify the values of this tensor. Higher level libraries like tf.keras use tf.Variable to store model parameters.

In [12]:
tensor1 = tf.constant([[1.0, 2.0], [3.0, 4.0]])
variable1 = tf.Variable(tensor1)

# Variables can be all kinds of types, just like tensors
boolean_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

A variable looks and acts like a tensor, and, in fact, is a data structure backed by a tf.Tensor. Like tensors, they have a dtype and a shape, and can be exported to NumPy

In [13]:
print("Shape: ",variable1.shape)
print("DType: ",variable1.dtype)
print("As Numpy: ",variable1.numpy)

Shape:  (2, 2)
DType:  <dtype: 'float32'>
As Numpy:  <bound method BaseResourceVariable.numpy of <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>>


Most tensor operations work on variables as expected, although variables cannot be reshaped

In [14]:
print("A variable: ", variable1)
print("\nViewed as a tensor: ", tf.convert_to_tensor(variable1))
print("\nIndex of high value: ", tf.math.argmax(variable1))


A variable:  <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

Viewed as a tensor:  tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)

Index of high value:  tf.Tensor([1 1], shape=(2,), dtype=int64)


This creates a new tensor; it does not reshape the variable.

In [15]:
print("nCopying and reshaping: ",tf.reshape(variable1, [1,4]))

nCopying and reshaping:  tf.Tensor([[1. 2. 3. 4.]], shape=(1, 4), dtype=float32)


As noted above, variables are backed by tensors. You can reassign the tensor using tf.Variable.assign. Calling assign does not (usually) allocate a new tensor; instead, the existing tensor's memory is reused.

In [16]:
b = tf.Variable([2.0, 3.0]) #keeps the same dtype,  float32
b.assign([2, 3])

<tf.Variable 'UnreadVariable' shape=(2,) dtype=float32, numpy=array([2., 3.], dtype=float32)>

If you use a variable like a tensor in operations, you will usually operate on the backing tensor.

Creating new variables from existing variables duplicates the backing tensors. Two variables will not share the same memory.

In [17]:
c = tf.Variable([2.0, 3.0])
d = tf.Variable(c) #creates d based on the value of c
c.assign([5, 6])
print(c.numpy()),print(d.numpy())

[5. 6.]
[2. 3.]


(None, None)

Other ways of variable assinment

In [18]:
print(c.assign_add([2,3]).numpy())
print(d.assign_add([8,8]).numpy())

[7. 9.]
[10. 11.]


https://www.tensorflow.org/guide/variable#placing_variables_and_tensors: Placing variables and tensors
For better performance, TensorFlow will attempt to place tensors and variables on the fastest device compatible with its dtype. This means most variables are placed on a GPU if one is available.

However, you can override this. In this snippet, place a float tensor and a variable on the CPU, even if a GPU is available. By turning on device placement logging (see Setup), you can see where the variable is placed.

In [19]:
with tf.device("CPU:0"):

  # Create some tensors
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
  c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [20]:
with tf.device("CPU:0"):
  a = tf.Variable([[1.0,2.0,3.0],[4.0,5.0,6.0]])
  b = tf.Variable([[1.0,2.0,3.0]])
with tf.device("GPU:0"):
  k = a * b # This is element-wise multiplication
  print(k)

tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)


# What we've created so far:
* Scalar: A single number
* Vector: A number with direction (e.g. speed, direction)
* Matrix: a 2-dimensional array of numbers
*Tensor: an n-dimensioanal array of numbers (where n can be any number, a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector )

tf.Variable Vs tf.constant

In [21]:
# Create the same tensor with tf.variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [23]:
# Let's try to change one of the elements in our  changeable tensor
changeable_tensor[0] = 7

TypeError: ignored

In [24]:
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [25]:
# Let's change the unchangeable_tensor
unchangeable_tensor[0] = 7

TypeError: ignored

In [26]:
unchangeable_tensor[0].assign(7)
unchangeable_tensor

AttributeError: ignored

### Creating randon tensors
Random tensors are tensors of some arbitrary size which contain random numbers

In [27]:
# create two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(42) # set seed for reproductivity
random_1 = random_1.normal(shape=(3, 2))
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3, 2))
# Are they equal?
random_1, random_2, random_1 == random_2

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

# Shuffle a tensor (valuable for when you want your data so the inherent order doesn't affect learning)

In [28]:
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])

In [29]:
not_shuffled.ndim

2

In [30]:
not_shuffled

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [31]:
# shuffle this non shufffled tensor
# we set the seed because we want a reproduceable tensor.
tf.random.set_seed(42) # Global random seed
tf.random.shuffle(not_shuffled, seed=42) # Operational-level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

In [32]:
# The above method to set-seed is not working. Details of global and operation- level seeds can be learned here...https://www.tensorflow.org/api_docs/python/tf/random/set_seed
#This is your assignment.
# In the code below, it seems that if we want to maintain the same order of the tensor content, we must use both Global and operation level random seed.
tf.random.set_seed(42) # Global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  4],
       [ 2,  5]], dtype=int32)>

# Other ways to make tensors

In [33]:
tf.ones([10, 7])

<tf.Tensor: shape=(10, 7), dtype=float32, numpy=
array([[1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1.]], dtype=float32)>

# Turn Numpy array into tensors
The main difference between Numpy arrays and tensorflow tensors is that tensors can be run on a GPU.


In [34]:
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # creates a numpy array between 1 and 25

In [35]:
numpy_A

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [36]:
A = tf.constant(numpy_A, shape=(3, 8))

In [37]:
A

<tf.Tensor: shape=(3, 8), dtype=int32, numpy=
array([[ 1,  2,  3,  4,  5,  6,  7,  8],
       [ 9, 10, 11, 12, 13, 14, 15, 16],
       [17, 18, 19, 20, 21, 22, 23, 24]], dtype=int32)>

In [38]:
A.ndim

2

In [39]:
B = tf.constant(numpy_A)

In [40]:
B

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [41]:
A.ndim

2

# Getting information fron tensors

In [56]:
#Useful resource: https://medium.com/mlearning-ai/what-are-tensors-495cf37c18e6

When dealing with tensors, you probably want to be aware of the following tensor attributes:
* shape
* Rank
* Axis or dimension
*size

In [45]:
#create a rank 4 tensor (4 dimensions)
rank_4 = tf.ones(shape=[3, 2, 4, 5])

In [46]:
rank_4

<tf.Tensor: shape=(3, 2, 4, 5), dtype=float32, numpy=
array([[[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]],


       [[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]],


       [[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]]], dtype=float32)>

In [47]:
rank_4[0]

<tf.Tensor: shape=(2, 4, 5), dtype=float32, numpy=
array([[[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]], dtype=float32)>

In [48]:
rank_4.shape

TensorShape([3, 2, 4, 5])

In [49]:
rank_4.ndim

4

In [50]:
tf.size(rank_4)

<tf.Tensor: shape=(), dtype=int32, numpy=120>

In [53]:
3 * 2 * 4 * 5

120

In [55]:
# Get various attributes of our tensor
print("Data type of every element: ", rank_4.dtype)
print("Number of dimensions (rank): ", rank_4.ndim)
print("shape of tensor: ", rank_4.shape)
print("Elements along the 0 axis: ", rank_4.shape[0])
print("Elements along the last axis: ", rank_4.shape[-1])
print("the total number of elements in our tensor: ", tf.size(rank_4).numpy())


Data type of every element:  <dtype: 'float32'>
Number of dimensions (rank):  4
shape of tensor:  (3, 2, 4, 5)
Elements along the 0 axis:  3
Elements along the last axis:  5
the total number of elements in our tensor:  120


# Indexing and Slicing tensors
Tensors can be indexed like python lists


In [58]:
some_lists = [1, 2, 3, 4]
some_lists[:2]

[1, 2]

In [59]:
some_lists[2:]

[3, 4]

In [60]:
rank_4[:2, :2, :2, :2]

<tf.Tensor: shape=(2, 2, 2, 2), dtype=float32, numpy=
array([[[[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]]],


       [[[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]]]], dtype=float32)>

In [61]:
# get the first element from each dimension from each index except for the last one

rank_4[1:, 1:, :, 1:]

<tf.Tensor: shape=(2, 1, 4, 4), dtype=float32, numpy=
array([[[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]],


       [[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]]], dtype=float32)>

In [62]:
# create rank 2 tensor
rank_2 = tf.constant([[10, 7],
                      [7, 10]])
rank_2

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [63]:
rank_2.shape, rank_2.ndim, tf.size(rank_2)

(TensorShape([2, 2]), 2, <tf.Tensor: shape=(), dtype=int32, numpy=4>)

In [64]:
# get the last item of each element in rank_2
rank_2

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [65]:
some_lists

[1, 2, 3, 4]

In [66]:
some_lists[3:]

[4]

In [68]:
# add in extra dimension to our rank 2 tensor
rank_3 = rank_2[..., tf.newaxis]
rank_3

<tf.Tensor: shape=(2, 2, 1), dtype=int32, numpy=
array([[[10],
        [ 7]],

       [[ 7],
        [10]]], dtype=int32)>

In [69]:
x = np.array([[[1, 4, 7],
               [2, 5, 8],
               [3, 6, 9]],
              [[10, 40, 70],
               [20, 50, 80],
               [30, 60, 90]],
              [[100, 400, 700],
               [200, 500, 800],
               [300, 600, 900]]])
print(x)
print('This tensor is of rank %d' %(x.ndim))

[[[  1   4   7]
  [  2   5   8]
  [  3   6   9]]

 [[ 10  40  70]
  [ 20  50  80]
  [ 30  60  90]]

 [[100 400 700]
  [200 500 800]
  [300 600 900]]]
This tensor is of rank 3


In [74]:
# There can be an arbitrary number of
# axes (sometimes called "dimensions")
rank_n = tf.constant([
  [[0, 1, 2, 3, 4],
   [5, 6, 7, 8, 9]],
  [[10, 11, 12, 13, 14],
   [15, 16, 17, 18, 19]],
  [[20, 21, 22, 23, 24],
   [25, 26, 27, 28, 29]],])

print(rank_n)

tf.Tensor(
[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]]

 [[10 11 12 13 14]
  [15 16 17 18 19]]

 [[20 21 22 23 24]
  [25 26 27 28 29]]], shape=(3, 2, 5), dtype=int32)


In [75]:
rank_n.shape

TensorShape([3, 2, 5])

In [76]:
a = tf.constant([[1, 2],
                 [3, 4]])

In [77]:
b = tf.constant([[1, 1],
                 [1, 1]])

In [78]:
a.shape

TensorShape([2, 2])

In [79]:
print(tf.add(a, b), "\n")

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32) 



In [80]:
print(tf.multiply(a, b), "\n")

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32) 



In [81]:
print(tf.matmul(a, b), "\n")

tf.Tensor(
[[3 3]
 [7 7]], shape=(2, 2), dtype=int32) 



In [82]:
c = tf.constant([[4.0, 5.0], [10.0, 1.0]])

# Find the largest value
print(tf.reduce_max(c))
# Find the index of the largest value
print(tf.math.argmax(c))
# Compute the softmax
print(tf.nn.softmax(c))

tf.Tensor(10.0, shape=(), dtype=float32)
tf.Tensor([1 0], shape=(2,), dtype=int64)
tf.Tensor(
[[2.6894143e-01 7.3105860e-01]
 [9.9987662e-01 1.2339458e-04]], shape=(2, 2), dtype=float32)


In [83]:
tf.convert_to_tensor([1,2,3])

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [84]:
rank_4_tensor = tf.zeros([4, 2, 4, 5])
rank_4_tensor

<tf.Tensor: shape=(4, 2, 4, 5), dtype=float32, numpy=
array([[[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]],


       [[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0

In [87]:
print("Number of axes:", rank_4_tensor.ndim)

Number of axes: 4


In [88]:
print("Shape of tensor:", rank_4_tensor.shape)

Shape of tensor: (4, 2, 4, 5)


In [89]:
print("Elements along axis 0 of tensor:", rank_4_tensor.shape[0])

Elements along axis 0 of tensor: 4


In [90]:
tf.rank(rank_4_tensor)

<tf.Tensor: shape=(), dtype=int32, numpy=4>

In [91]:
tf.shape(rank_4_tensor)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([4, 2, 4, 5], dtype=int32)>

In [92]:
# create a (3, 2) tensor
X = tf.constant([[2, 4],
                [3, 5],
                [4, 6]])


#creating another (3, 2) tensor
Y = tf.constant([[2,3],
                 [1, 1],
                 [4, 8]])

In [93]:
X, Y

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[2, 4],
        [3, 5],
        [4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[2, 3],
        [1, 1],
        [4, 8]], dtype=int32)>)

In [96]:
#Try to multiply matrix of the same shape
tf.multiply(X, Y)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[ 4, 12],
       [ 3,  5],
       [16, 48]], dtype=int32)>

In [97]:
#change Y SHAPE
tf.reshape(Y,shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[2, 3, 1],
       [1, 4, 8]], dtype=int32)>

In [98]:
X @ tf.reshape(Y, shape=(2, 3))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 8, 22, 34],
       [11, 29, 43],
       [14, 36, 52]], dtype=int32)>

In [99]:
tf.matmul(X, tf.reshape(Y,shape=(2,3)))

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[ 8, 22, 34],
       [11, 29, 43],
       [14, 36, 52]], dtype=int32)>

In [100]:
tf.matmul(tf.reshape(X, shape=(2,3)),Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[20, 34],
       [38, 67]], dtype=int32)>

In [101]:
X, tf.transpose(X),tf.reshape(X,shape=(2,3))

(<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
 array([[2, 4],
        [3, 5],
        [4, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[2, 3, 4],
        [4, 5, 6]], dtype=int32)>,
 <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
 array([[2, 4, 3],
        [5, 4, 6]], dtype=int32)>)

In [102]:
#Try matrix multiplication rather than reshape
tf.matmul(tf.transpose(X),Y)

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[23, 41],
       [37, 65]], dtype=int32)>